# Lab 5

## Import Libraries:

First, import the necessary libraries:

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets

## Prepare the Data:

Download and prepare a dataset. For example, we can use the MNIST dataset for handwritten digit recognition:

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

## Create the CNN Model:

Define your CNN architecture by creating a custom neural network class that inherits from nn.Module. Here's an example:

In [3]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3).cuda()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3).cuda()
        self.fc1 = nn.Linear(64 * 5 * 5, 128).cuda()
        self.fc2 = nn.Linear(128, 10).cuda()
    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2)
        x = x.view(-1, 64 * 5 * 5)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## Instantiate the Model and Define Loss Function & Optimizer:

Instantiate your CNN model and define the loss function and optimizer:

In [4]:
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Training Loop:

Train the model using the training data:

In [5]:
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:
            print(f'Epoch [{epoch + 1}/{num_epochs}], '
                  f'Step [{i + 1}/{len(train_loader)}], '
                  f'Loss: {running_loss / 100:.4f}')
            running_loss = 0.0

Epoch [1/5], Step [100/938], Loss: 0.6131
Epoch [1/5], Step [200/938], Loss: 0.1514
Epoch [1/5], Step [300/938], Loss: 0.1181
Epoch [1/5], Step [400/938], Loss: 0.0949
Epoch [1/5], Step [500/938], Loss: 0.0973
Epoch [1/5], Step [600/938], Loss: 0.0753
Epoch [1/5], Step [700/938], Loss: 0.0741
Epoch [1/5], Step [800/938], Loss: 0.0585
Epoch [1/5], Step [900/938], Loss: 0.0573
Epoch [2/5], Step [100/938], Loss: 0.0588
Epoch [2/5], Step [200/938], Loss: 0.0453
Epoch [2/5], Step [300/938], Loss: 0.0414
Epoch [2/5], Step [400/938], Loss: 0.0410
Epoch [2/5], Step [500/938], Loss: 0.0428
Epoch [2/5], Step [600/938], Loss: 0.0522
Epoch [2/5], Step [700/938], Loss: 0.0431
Epoch [2/5], Step [800/938], Loss: 0.0398
Epoch [2/5], Step [900/938], Loss: 0.0406
Epoch [3/5], Step [100/938], Loss: 0.0310
Epoch [3/5], Step [200/938], Loss: 0.0300
Epoch [3/5], Step [300/938], Loss: 0.0336
Epoch [3/5], Step [400/938], Loss: 0.0360
Epoch [3/5], Step [500/938], Loss: 0.0239
Epoch [3/5], Step [600/938], Loss:

1m 21.2s - CUDA

## Testing the Model:

Evaluate the model using the test data:

In [6]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test set: {(100 * correct / total):.2f}%')

Accuracy on the test set: 99.08%


2.3s - CUDA